# 参考mnist_conv-keras实现针对cifar10的卷积模型

## 准备数据

In [31]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import numpy as np
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def cifar10_dataset():
    (x, y), (x_test, y_test) = datasets.cifar10.load_data()
    y=y.reshape(-1)
    y_test=y_test.reshape(-1)
    train_label = np.zeros(shape=[y.shape[0], 10])
    test_label = np.zeros(shape=[y_test.shape[0], 10])
    train_label[np.arange(y.shape[0]), np.array(y)] = 1.
    test_label[np.arange(y_test.shape[0]), np.array(y_test)] = 1.
    
    ds = tf.data.Dataset.from_tensor_slices((x, train_label))
    ds = ds.map(prepare_cifar10_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    
    test_ds = tf.data.Dataset.from_tensor_slices((x_test, test_label))
    test_ds = test_ds.map(prepare_cifar10_features_and_labels)
    test_ds = test_ds.take(20000).shuffle(20000).batch(20000)
    return ds, test_ds

def prepare_cifar10_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

In [36]:
train_ds, test_ds = cifar10_dataset()
for x,y in iter(train_ds):
    break
x.shape,y.shape


(TensorShape([100, 32, 32, 3]), TensorShape([100, 10]))

In [37]:
x.shape,y[0]

(TensorShape([100, 32, 32, 3]),
 <tf.Tensor: id=9477, shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=int64)>)

In [33]:
(x, y), (x_test, y_test) = datasets.cifar10.load_data()

In [34]:
y.reshape(-1)

array([6, 9, 9, ..., 9, 1, 1], dtype=uint8)

In [35]:
x.shape,y[0]

((50000, 32, 32, 3), array([6], dtype=uint8))

## 建立模型

In [38]:
model = keras.Sequential([
    Conv2D(32, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Conv2D(64, (5, 5), activation='relu', padding='same'),
    MaxPooling2D(pool_size=2, strides=2),
    Flatten(), 
    layers.Dense(128, activation='relu'), #N*128
    layers.Dense(10,activation='softmax')]) #N*10
optimizer = optimizers.Adam()

## 编译， fit以及evaluate

In [39]:
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
train_ds, test_ds = cifar10_dataset()

In [40]:
model.fit(train_ds, epochs=5)
model.evaluate(test_ds)

Epoch 1/5
200/200 [==============================] - 3s 13ms/step - loss: 1.8188 - accuracy: 0.2557
Epoch 2/5
200/200 [==============================] - 2s 11ms/step - loss: 1.4681 - accuracy: 0.4590
Epoch 3/5
200/200 [==============================] - 2s 11ms/step - loss: 1.3095 - accuracy: 0.5225
Epoch 4/5
200/200 [==============================] - 2s 11ms/step - loss: 1.1830 - accuracy: 0.5683
Epoch 5/5
1/1 [==============================] - 0s 465ms/step - loss: 1.1236 - accuracy: 0.6085


[1.1235620975494385, 0.6085]

In [41]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           multiple                  2432      
_________________________________________________________________
max_pooling2d_10 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_11 (Conv2D)           multiple                  51264     
_________________________________________________________________
max_pooling2d_11 (MaxPooling multiple                  0         
_________________________________________________________________
flatten_5 (Flatten)          multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  524416    
_________________________________________________________________
dense_11 (Dense)             multiple                 